In [86]:
import pickle
import sys
sys.path.append('')
sys.path.append('../')
sys.path.append('../../data/virtualTimetable/')
sys.path.append('/home/step/data/xcl/JDBUSgroup')
with open('/home/step/data/xcl/JDBUSgroup/data/virtualTimetable/timetableSeqGroundTruth.pkl','rb') as f:
    data = pickle.load(f)

# 生成的data的应该是按照字典顺序排列的
assert isinstance(data,dict)

# 返回结果的形式应该是
    # res={
    #     'timetableChain':timetableChain,
    #     'headwayPoint':headwayPoint,
    #     'headwayTime':headwayTime,
    #     'runningPoint':runningPoint,
    #     'runningTime':runningTime,
    #     'mealInterval':mealInterval,
    #     'mealCount':mealCount,
    #     'workmodeInterval':workmodeInterval,
    #     'workmodeCount':workmodeCount
    # }
len(data)

16

In [87]:
index = [i for i in data]
print(index[5])
test =  data[index[5]]['seqTimetable']

嘉定56路weekend


In [88]:
print(test)
# 解析生成timetable array,根据[depature time,arrival time,index,pattern,workmode]来得到长序列

     depature time  direction  index  running time
0              390          0      1            35
1              430          1      1            50
2              495          0      1            40
3              540          1      1            40
4              615          0      1            40
..             ...        ...    ...           ...
132            765          1     10            40
133           1062          0     10            40
134           1107          1     10            45
135           1185          0     10            35
136           1225          1     10            30

[137 rows x 4 columns]


In [89]:
import torch
import torch.nn as nn 
import numpy as np
from  torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
myArray = np.array(test)
myArray = myArray[myArray[:,0].argsort()]
myArray[:,0] = myArray[:,0]/3600
myArray[:,-1] = myArray[:,-1]/60
rawDataset = torch.tensor(myArray,dtype = torch.float)

numsteps = 4
length = len(rawDataset)
features = torch.zeros(length-numsteps,numsteps,2)
for i in range(numsteps):
    features[:,i,:] = rawDataset[i:length-numsteps+i,[0,3]]
targets = rawDataset[numsteps:,[0,3]]

seqdataset = TensorDataset(features,targets)

batchsize = 16
data_iter = DataLoader(seqdataset,batch_size = batchsize,shuffle = True)


In [90]:
for X,y in data_iter:
    print(X.shape)
    print(y.shape)

torch.Size([16, 4, 2])
torch.Size([16, 2])
torch.Size([16, 4, 2])
torch.Size([16, 2])
torch.Size([16, 4, 2])
torch.Size([16, 2])
torch.Size([16, 4, 2])
torch.Size([16, 2])
torch.Size([16, 4, 2])
torch.Size([16, 2])
torch.Size([16, 4, 2])
torch.Size([16, 2])
torch.Size([16, 4, 2])
torch.Size([16, 2])
torch.Size([16, 4, 2])
torch.Size([16, 2])
torch.Size([5, 4, 2])
torch.Size([5, 2])


In [91]:
class net(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.layer1 = nn.Linear(2,10)
        self.layer2 = nn.Linear(10,1)
    
    def forward(self,X):
        '''X 的输入4*4'''
        y1 = nn.functional.relu(self.layer1(X))
        return self.layer2(y1)

def init_weigt(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

model = net()
model.apply(init_weigt)

net(
  (layer1): Linear(in_features=2, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=1, bias=True)
)

In [93]:
loss = nn.MSELoss(reduce='none')
opt = torch.optim.Adam(model.parameters(),lr=0.03)
epoches = 100
device = torch.device('cuda')
model = model.to(device)
for epoch in range(epoches):
    for X,y in data_iter:
        X,y = X.to(device),y.to(device)
        opt.zero_grad()
        y_hat = model(X)
        y_hat = y_hat.squeeze(-1)
        print(y_hat.shape)
        l = loss(y_hat,y)
        l.mean().backward()
        opt.step()
        print(l.mean())

torch.Size([16, 4])


/home/step/anaconda3/envs/xclds/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/home/step/anaconda3/envs/xclds/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([16, 2])) that is different to the input size (torch.Size([16, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (4) must match the size of tensor b (2) at non-singleton dimension 1